In [1]:
from bs4 import BeautifulSoup
import requests
import pickle
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import numpy as np

In [2]:
def active_cases(s):
    s.loc["Active cases"]=str(int(s.loc["Coronavirus Cases"].replace(',',''))-int(s.loc["Deaths"].replace(',',''))-int(s.loc["Recovered"].replace(',','')))
    count=0
    number=''
    for digit in s.loc["Active cases"][::-1]:
        if count==3:
            number+=','
            count=0
        number+=digit
        count+=1
    s.loc["Active cases"]=number[::-1]
    return s

def dump_obj(path,obj):
    f=open(path,'wb')
    pickle.dump(obj,f)
    f.close()

In [3]:
#Real time data : Summary of cases , World-wide news ,WHO news

website='https://pandemic.internationalsos.com/2019-ncov/covid-19-daily-summary'
source=requests.get(website).text
soup=BeautifulSoup(source,'html.parser')
covid_summary=soup.find('div',{'class':'module freetext large var1 bodycontent'}).find('div',{'class':'wrapper cf'})

for index in range(0,len(covid_summary.find_all('p'))-5):
    try:
        if index==0:
            published_time=covid_summary.find_all('p')[index].text
        elif index==1:
            details=covid_summary.find_all('p')[index].text
        elif index==4:
            who_news=covid_summary.find_all('p')[index].text+covid_summary.find_all('p')[index+1].text
            countrylist=['WHO Summary']
            newslist=[who_news]
        elif index>=6:
            country,news=covid_summary.find_all('p')[index].text.split(':')
            countrylist.append(country)
            newslist.append(news)
    except:
        pass
newsdf=pd.DataFrame({'Country':countrylist,'News':newslist})

index=['Last updated']
data=[published_time.split('\n')[0]]
for i in (details.split('\n')):
    a,b=i.split(':')
    index.append(a)
    data.append(b.strip())
s=pd.Series(data,index)

dump_obj('covid_summary.pkl',s)

dump_obj('countrywise_news.pkl',newsdf)

#print(s)
#print(newsdf)

In [4]:
# Mythbusters

who_homepage="https://www.who.int/emergencies/diseases/novel-coronavirus-2019"
source1=requests.get(who_homepage+'/advice-for-public/myth-busters').text
soup1=BeautifulSoup(source1,'lxml')
mythbusters=soup1.find_all('div',{'class':'sf-content-block content-block'})
d={'Headlines':[],'Content':[]}
for mythbuster in mythbusters:
    try:
        headline=mythbuster.div.h2.text
        content=mythbuster.div.text
        d['Headlines'].append(headline)
        d['Content'].append(content)
    except:
        pass
mythbuster_df=pd.DataFrame(d)

#dump_obj('D:\\Jupyter notebooks\\Cods summer project 2020_Chatbot\\Static Data\\mythbusters.pkl',mythbuster_df)

#print(mythbuster_df)

In [9]:
#Covid-19 detailed realtime updates country_wise

link='https://www.worldometers.info/coronavirus/'
source2=requests.get(link).text
soup2=BeautifulSoup(source2,'lxml')
rows=soup2.find('table',{'id':'main_table_countries_today'})
column_names=["Country1"]
ths=rows.find_all('th')
for th in ths:
    if th.text.strip()!='#':
        column_names.append(th.text.replace('\n',''))
rows=rows.tbody.find_all("tr")
d=dict()
country_links={}
for index in range(len(rows)):
    try:
        countryinfo=rows[index].find_all('a',href=True)[0]
        country=countryinfo.text
        country_link=countryinfo['href']
        country_links[country]=link+country_link
        data=[i.text for i in rows[index].find_all('td')]+[link+country_link]
    except:
        country=rows[index].find_all('td')[0].text
        data=[i.text for i in rows[index].find_all('td')]+[np.NaN]
    d[country]=data

column_names=column_names+["Link for more about the country"]
realtime_data=pd.DataFrame(d).iloc[1:,:].T.reset_index()
realtime_data.columns=column_names
del realtime_data['Country1']


dump_obj('worldwide_statistics.pkl',realtime_data)
    
#print(realtime_data)

In [10]:
#USA statewise
link=list(country_links.values())[0]
source=requests.get(link).text
soup=BeautifulSoup(source,'lxml')
rows=soup.find('table',{'id':'usa_table_countries_today'})
rows1=rows.find_all('tbody')[0].find_all("tr")
rows2=rows.find_all('tbody')[1].find_all("tr")
rows=rows1+rows2
state_d={}
state_links={}
for index in range(len(rows)):
    try:
        stateinfo=rows[index].find_all('a',href=True)[0]
        state_link=stateinfo['href']
        if state_link.startswith('http'):
            data=[i.text.strip() for i in row[index].find_all('td')]+[state_link]
        else:
            data=[i.text.strip() for i in rows[index].find_all('td')]+[link[:-1]+state_link]
        state=data[0]
        if state_link.startswith('http'):
            state_links[state]=state_link
        else:
            state_links[state]=link[:-1]+state_link
    except:
        data=[i.text.strip() for i in rows[index].find_all('td')]+[np.NaN]
        state=data[0]
    state_d[state]=data
     

column_names=["State","Total Cases","New Cases", "Total Deaths", "New Deaths","Active Cases","Total Cases/1M population","Deaths/1M population","Total tests","Total tests/1M population",'a','b',"Link for more about the state"]
usa_state_data=pd.DataFrame(state_d).iloc[1:,:].T.reset_index()
usa_state_data.columns=column_names
del usa_state_data['a']
del usa_state_data['b']

dump_obj('USA_state_statistics.pkl',usa_state_data)

#print(usa_state_data)
#print(state_links)

    

In [11]:
#USA county-wise data

for state_link in state_links:
        link=state_links[state_link]
        source=requests.get(link).text
        soup=BeautifulSoup(source,'lxml')
        try:
            rows=soup.find('table',{'id':'usa_table_countries_today'})
            rows=rows.find_all('tbody')[0].find_all("tr")
            county_d={}
            for index in range(len(rows)):
                    data=[i.text.strip() for i in rows[index].find_all('td')]
                    county=data[0]
                    county_d[county]=data

            column_names=["County","Total Cases","New Cases", "Total Deaths", "New Deaths","Active Cases","Total tests",'a']
            usa_county_data=pd.DataFrame(county_d).iloc[1:,:].T.reset_index()
            usa_county_data.columns=column_names
            del usa_county_data['a']

            dump_obj("D:\\Jupyter notebooks\\Cods summer project 2020_Chatbot\\USA_countywise_statistics\\"+state_link+'_statistics.pkl',usa_county_data)
        except:
            county_d={}
            rows=soup.find_all("div",{'id':'maincounter-wrap'})
            for row in rows[:-1]:
                title=row.h1.text.strip()[:-1]
                data=row.div.text.strip()
                county_d[title]=data
            county_s=pd.Series(county_d)
            county_s=active_cases(county_s)
                
            dump_obj("D:\\Jupyter notebooks\\Cods summer project 2020_Chatbot\\USA_countywise_statistics\\"+state_link+'_statistics.pkl',county_s)

                


In [12]:
#realtime covid-19 summary

link='https://www.worldometers.info/coronavirus/'
source=requests.get(link).text
soup=BeautifulSoup(source,'lxml')
divs=soup.find_all('div',{'id':'maincounter-wrap'})
d={}
for div in divs:
    title=div.h1.text[:-1]
    data=div.span.text
    d[title]=data
box=soup.find('div',{'class':'col-md-6'})
data1=box.find('div',{'style':'float:left; text-align:center'}).span.text
title1=box.find('div',{'style':'float:left; text-align:center'}).div.text
data2=box.find('div',{'style':'float:right; text-align:center'}).span.text
title2=box.find('div',{'style':'float:right; text-align:center'}).div.text
d[title1]=data1
d[title2]=data2
s=pd.Series(d)
active_cases(s)
dump_obj('covid_summary_realtime.pkl',s)

In [13]:
#Covid 19 incubation period

new_link='https://www.worldometers.info/coronavirus/coronavirus-incubation-period/'
source=requests.get(new_link).text
soup=BeautifulSoup(source,'lxml')
d={'Heading':[],'Content':[]}
d['Heading'].append(soup.find('h4',{'id':'incubation-estimates'}).text.strip())
d['Heading'].append(soup.find('h3').text.strip())
d['Heading'].append(soup.find('h4',{'id':'24'}).text.strip())
for ul in soup.find_all('ul')[1:-2]:
    d['Content'].append(ul.text.strip())
d1={'Virus':[],'Incubation period':[]}
td=soup.find_all('td')
for i in td:
    if i['align']=='left':
        d1['Virus'].append(i.text.strip())
    else:
        d1['Incubation period'].append(i.text.replace('\n',''))
df1=pd.DataFrame(d1)
d['Heading'].append('Comparission Table')
d['Content'].append(df1)
df=pd.DataFrame(d)

dump_obj('D:\\Jupyter notebooks\\Cods summer project 2020_Chatbot\\Static Data\\incubation_period.pkl',df)

In [14]:
#Q/A about the virus by WHO

qna_link='https://www.who.int/emergencies/diseases/novel-coronavirus-2019/question-and-answers-hub'
source=requests.get(qna_link).text
soup=BeautifulSoup(source,'lxml')
q_links={}
for q in zip(soup.find_all('span',{'class':'trimmed'}),soup.find_all('a',{'class':'sf-list-vertical__item'})):
    q_links[q[0].text]='https://www.who.int'+q[1]['href']
for q in q_links:
        new_source=requests.get(q_links[q]).text
        new_soup=BeautifulSoup(new_source,'lxml')
        qna={'Questions':[],'Answers':[]}
        question=new_soup.find('div',{'class':'qa-details__content'}).find('div',{'class':'col-md-7'}).find_all('a',{'class':'sf-accordion__link'})
        answer=new_soup.find('div',{'class':'qa-details__content'}).find('div',{'class':'col-md-7'}).find_all('div',{'class':'sf-accordion__content'})
        for que in zip(question,answer):
            qna['Questions'].append(que[0].text.strip())
            qna['Answers'].append(que[1].text.strip())
        df=pd.DataFrame(qna)
        
        #dump_obj("D:\\Jupyter notebooks\\Cods summer project 2020_Chatbot\\Q and A\\"+q.replace(':',' on').replace('/',' OR ')+'.pkl',df)
        
    

In [15]:
#Preventive measures , Masks , sanitizers:

link='http://who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public'
source=requests.get(link).text
soup=BeautifulSoup(source,'lxml')
measures={}
heading=soup.find_all('div',{'class':"section-heading"})[3:1:-1]
m=soup.find('div',{'class':'sf_colsIn col-md-10'}).find_all('div',{'class':'sf-content-block content-block'})[2:4]
for i in range(len(m)):
    measures[heading[i].text.strip()]=m[i].text.strip().split('.')[:-1]

link2='https://www.who.int/emergencies/diseases/novel-coronavirus-2019/advice-for-public/when-and-how-to-use-masks'
source2=requests.get(link2).text
soup2=BeautifulSoup(source2,'lxml')
heading2=soup2.find('div',{'class':'sf_colsIn col-md-9'}).find_all('div',{'class':'section-heading'})[:2]
m2=soup2.find('div',{'class':'sf_colsIn col-md-9'}).find_all('div',{'class':'sf-content-block content-block'})[:2]
for i in range(len(m2)):
    measures[heading2[i].text.strip()]=m2[i].text.replace('&amp;amp;amp;amp;nbsp;','').strip().split('.')[:-1]

max_size=max([len(measures[x]) for x in measures])
for i in measures:
    if(len(measures[i])!=max_size):
        measures[i]+=(['']*(max_size-len(measures[i])))

df=pd.DataFrame(measures)

#dump_obj('D:\\Jupyter notebooks\\Cods summer project 2020_Chatbot\\Static Data\\Protective_measures(masks,sanitizers etc).pkl',df)


In [20]:
#rolling updates on COVID-19 by WHO:

link='https://www.who.int/emergencies/diseases/novel-coronavirus-2019/events-as-they-happen'
source=requests.get(link).text
soup=BeautifulSoup(source,'lxml')
news={'Date':[],'Headline':[],'News':[]}
topics=soup.find_all('div',{'class':'sf-content-block content-block'})
for topic in topics:
    try:
        date=topic.h2
    except:
        date='31 December 2019'
    headline=topic.find('span',{'style':'background-color:transparent;font-size:25px;font-weight:700;text-align:inherit;white-space:inherit;word-spacing:normal;caret-color:auto;'})
    try:
        print(date.text,headline.text)
    except:
        pass

 WHO, Gavi and UNICEF warn that disruption to routine vaccination leaves at least 80 million children at risk  


In [ ]:
#f=open('.pkl','rb')
#print(pickle.load(f))
#f.close()
